In [1]:
import tensorflow as tf
from Preprocess import label_for_nn
from vgg11_model import model

In [2]:
benign_path = '/home/user01/Desktop/DDSM/4_benign_Pleo_10_ROI_subpic'#input('良性子图路径')#
cancer_path = '/home/user01/Desktop/DDSM/4_cancer_Pleo_10_ROI_subpic'#input('恶性子图路径')#
saver_path = '/home/user01/Desktop/DDSM/model/VGG11'#nput('模型保存路径')#
size = 128
tf.reset_default_graph()

In [3]:
def get_trainsets_and_testsets(benign_path,cancer_path):
    (train_x ,train_y),(test_x,test_y) = label_for_nn(benign_path,cancer_path,scale=0.2,kind='RGB')
    train_x = train_x.reshape(train_x.shape[0],size,size,3).astype('float32')/255.0
    test_x = test_x.reshape(test_x.shape[0],size,size,3).astype('float32')/255.0
    return  test_x ,test_y

In [4]:
def conv2d_P(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='SAME')

In [5]:
def conv2d(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='VALID')

In [6]:
def max_pooling2x2(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')

In [7]:
def cnn_test():
    x = tf.placeholder(tf.float32, [None, 128, 128, 3])
    y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)
    y_pre = model(x,keep_prob)
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pre), reduction_indices=[1]))
    y_pre_label = tf.argmax(y_pre,1)
    y_label = tf.argmax(y,1)
    a = tf.add(y_pre_label, 1)
    b = tf.add(y_label, 3)
    c = tf.multiply(a, b)
    tp = tf.reduce_sum(tf.cast(tf.equal(c, 8), tf.float32))
    fp = tf.reduce_sum(tf.cast(tf.equal(c, 6), tf.float32))
    tn = tf.reduce_sum(tf.cast(tf.equal(c, 3), tf.float32))
    fn = tf.reduce_sum(tf.cast(tf.equal(c, 4), tf.float32))
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    tf.summary.scalar('test_Sensitivity', tpr)
    tf.summary.scalar('test_Specificity', tnr)
    correct_l1 = tf.equal(y_pre_label, y_label)
    accuracy = tf.reduce_mean(tf.cast(correct_l1, tf.float32))
    tf.summary.scalar('test_loss', cross_entropy)
    tf.summary.scalar('test_accuracy', accuracy)
    merged_summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        test_accuracy = 0
        writer = tf.summary.FileWriter(saver_path, sess.graph)
        saver.restore(sess,saver_path+'/VGG11')
        test_accuracy = 0
        test_sensitivity = 0
        test_specificity = 0
        test_loss = 0
        for n in range(0,100):
            test_x, test_y = get_trainsets_and_testsets(benign_path,cancer_path)
            acc,sen,spec,loss,summary = sess.run([accuracy,tpr,tnr,cross_entropy,merged_summary_op],feed_dict={x:test_x,y:test_y,keep_prob:1.0})
            test_accuracy += acc
            test_sensitivity += sen
            test_specificity +=spec
            test_loss += loss
            print("{0:3d} accuracy:{1:>10.9f} loss:{2:>10.9f} specificity:{3:>10.9f} sensitivity:{4:>10.9f}".format(n,acc,loss,sen,spec))
            writer.add_summary(summary=summary, global_step=n)
        print("accuracy:{0:<6.3f}% loss:{1:<6.3f}% specificity:{2:<6.3f}% sensitivity:{3:<6.3f}%".format(test_accuracy,test_loss,test_sensitivity,test_specificity))

In [8]:
cnn_test()

INFO:tensorflow:Restoring parameters from /home/user01/Desktop/DDSM/model/VGG11/VGG11
  0 accuracy:0.862068951 loss:0.310820669 specificity:0.888888896 sensitivity:0.838709652
  1 accuracy:0.833333313 loss:0.422951519 specificity:0.937500000 sensitivity:0.769230783
  2 accuracy:0.939999998 loss:0.178974345 specificity:1.000000000 sensitivity:0.896551728
  3 accuracy:0.780487776 loss:0.452377141 specificity:0.937500000 sensitivity:0.680000007
  4 accuracy:0.909090936 loss:0.239848197 specificity:1.000000000 sensitivity:0.833333313
  5 accuracy:0.787234068 loss:0.376538754 specificity:0.916666687 sensitivity:0.652173936
  6 accuracy:0.844827592 loss:0.394608915 specificity:0.968750000 sensitivity:0.692307711
  7 accuracy:0.875000000 loss:0.313325316 specificity:0.933333337 sensitivity:0.807692289
  8 accuracy:0.906976759 loss:0.209082067 specificity:1.000000000 sensitivity:0.818181813
  9 accuracy:0.869565189 loss:0.284257144 specificity:0.959999979 sensitivity:0.761904776
 10 accuracy:0

 91 accuracy:0.823529422 loss:0.463499367 specificity:0.954545438 sensitivity:0.724137902
 92 accuracy:0.943396211 loss:0.128518969 specificity:0.962962985 sensitivity:0.923076928
 93 accuracy:0.980769217 loss:0.099281944 specificity:0.958333313 sensitivity:1.000000000
 94 accuracy:0.893617034 loss:0.348284751 specificity:1.000000000 sensitivity:0.821428597
 95 accuracy:0.862745106 loss:0.259926677 specificity:0.962962985 sensitivity:0.750000000
 96 accuracy:0.842105269 loss:0.406601220 specificity:0.937500000 sensitivity:0.720000029
 97 accuracy:0.862745106 loss:0.271367759 specificity:0.925925910 sensitivity:0.791666687
 98 accuracy:0.823529422 loss:0.441704810 specificity:0.875000000 sensitivity:0.777777791
 99 accuracy:0.980392158 loss:0.133507609 specificity:1.000000000 sensitivity:0.956521749
accuracy:88.269% loss:27.817% specificity:95.683% sensitivity:80.816%
